In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('Classeur.csv')

In [37]:
df

,Gas,mmBtu per unit,kg CO2 per mmBtu,g CH4 per mmBtu,g N20 per mmBtu,kg CO2 per unit,kg CH4 per unit,kg N2O per unit,Unnamed: 8
0,Asphalt and Road Oil,0.158000,75.36,3.0,0.60,11.91,0.47,0.090000,gallon
1,Aviation Gasoline,0.120000,69.25,3.0,0.60,8.31,0.36,0.070000,gallon
2,Butane,0.103000,64.77,3.0,0.60,6.67,0.31,0.060000,gallon
3,Butylene,0.105000,68.72,3.0,0.60,7.22,0.32,0.060000,gallon
4,Crude Oil,0.138000,74.54,3.0,0.60,10.29,0.41,0.080000,gallon
...,...,...,...,...,...,...,...,...,...
60,Fuel Gas,0.001388,59.00,3.0,0.60,0.08189,0.004164,0.000833,scf
61,Pro ane Gas,0.002516,61.46,0.022,0.10,0.15463,0.000055,0.000252,scf
62,Biomass Fuels Gaseous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Landfill Gas,0.000485,52.07,3.2,0.63,0.025254,0.001552,0.000306,scf


In [38]:
df.rename(columns={'kg CH4 per unit': 'g CH4 per unit'}, inplace=True)
df.rename(columns={'kg N2O per unit': 'g N2O per unit'}, inplace=True)
df.rename(columns={'Unnamed: 8': 'unit'}, inplace=True)

In [39]:
df.dropna(inplace=True)

In [40]:
df

,Gas,mmBtu per unit,kg CO2 per mmBtu,g CH4 per mmBtu,g N20 per mmBtu,kg CO2 per unit,g CH4 per unit,g N2O per unit,unit
0,Asphalt and Road Oil,0.158000,75.36,3.0,0.60,11.91,0.47,0.090000,gallon
1,Aviation Gasoline,0.120000,69.25,3.0,0.60,8.31,0.36,0.070000,gallon
2,Butane,0.103000,64.77,3.0,0.60,6.67,0.31,0.060000,gallon
3,Butylene,0.105000,68.72,3.0,0.60,7.22,0.32,0.060000,gallon
4,Crude Oil,0.138000,74.54,3.0,0.60,10.29,0.41,0.080000,gallon
5,Distillate Fuel Oil No. 1,0.139000,73.25,3.0,0.60,10.18,0.42,0.080000,gallon
6,Distillate Fuel Oil No. 2,0.138000,73.96,3.0,0.60,10.21,0.41,0.080000,gallon
7,Distillate Fuel Oil No. 4,0.146000,75.04,3.0,0.60,10.96,0.44,0.090000,gallon
8,Ethane,0.068000,59.60,3.0,0.60,4.05,0.20,0.040000,gallon
9,Ethylene,0.058000,65.96,3.0,0.60,3.83,0.17,0.030000,gallon


In [41]:
df = df.replace(',', '.', regex=True)

In [42]:
C02_factor = 1
CH4_factor = 25
N2O_factor = 298

In [43]:
gallon_conversion = 3.78541
scf_conversion = 28.316846592
short_tons_conversion = 909.09

In [45]:
import json
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Charger le fichier JSON
with open('carbonfootprint_ssh/data/sources.json') as json_file:
    data = json.load(json_file)


# Parcourir chaque élément du fichier JSON
for element in data:
    # Vérifier si l'élément a des émissions de CO2, CH4 et N2O
    if 'emissions' in element and 'co2' in element['emissions'] and 'ch4' in element['emissions'] and 'n2o' in element['emissions']:
        # Rechercher le nom de l'élément dans la colonne "Gas" du DataFrame
        gas_name = element['name']

        # Utiliser FuzzyWuzzy pour trouver la correspondance la plus proche
        matches = process.extractOne(gas_name, df['Gas'], scorer=fuzz.partial_ratio)
        matched_gas_name = matches[0]
        similarity_score = matches[1]

        # Définir une limite de similarité pour considérer une correspondance valide
        similarity_threshold = 80

        if similarity_score >= similarity_threshold:
            # Obtenir la ligne correspondante dans le DataFrame
            row = df[df['Gas'] == matched_gas_name]

            # Récupérer les données du DataFrame
            co2_per_unit = float(row['kg CO2 per unit'].values[0])
            ch4_per_unit = float(row['g CH4 per unit'].values[0])
            n2o_per_unit = float(row['g N2O per unit'].values[0])
            unit = row['unit'].values[0]

            # Calculer la valeur de CO2e
            if unit == 'gallon':
                unit_convertor = gallon_conversion
            elif unit == 'sfc':
                unit_convertor = scf_conversion
            elif unit == 'short tons':
                unit_convertor = short_tons_conversion
            else:
                unit_convertor = 1.0
    
            co2e_value = (co2_per_unit * C02_factor +
                         ch4_per_unit * CH4_factor +
                         n2o_per_unit * N2O_factor) / unit_convertor

            # Ajouter la clé "co2e" à l'élément JSON
            element['emissions']['co2e'] = {
                'base_unit': 'kg',
                'value': co2e_value,
                'unit': 'kg',
                'reference_url': 'your_reference_url'
            }

# Enregistrer le fichier JSON modifié
with open('sources.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)


In [46]:

# Charger l'ancien fichier JSON
with open('carbonfootprint_ssh/data/sources.json') as old_file:
    old_data = json.load(old_file)

# Charger le nouveau fichier JSON
with open('sources.json') as new_file:
    new_data = json.load(new_file)

# Parcourir les éléments des deux fichiers JSON
compteur = 0
for old_element, new_element in zip(old_data, new_data):
    old_has_co2e = 'co2e' in old_element.get('emissions', {})
    new_has_co2e = 'co2e' in new_element.get('emissions', {})

    if not old_has_co2e and new_has_co2e:
        compteur +=1
        print(new_element['name'])

print ("\n Il y a ",compteur,"nouvelles données")

Crude oil
Natural Gas Liquids
Aviation gasoline
Jet gasoline
Jet kerosene
Other kerosene
Residual fuel oil
Liquified Petroleum Gases
Ethane
Naphtha
Bitumen
Lubricants
Petroleum coke
Anthracite
Other bituminous coal
Sub bituminous coal
Lignite
Coke oven gas
Blast furnace gas
Oxygen steel furnace gas
Natural gas
Biogasoline
Biodiesels
Landfill gas
Other biogas
Peat

 Il y a  26 nouvelles données
